# Resolution of the elastic problem of a sphere in an infinite matrix

In [1]:
using TensND, LinearAlgebra, SymPy, Tensors, OMEinsum, Rotations
sympy.init_printing(use_unicode=true)

## Definition of the coordinate system, base vectors...

In [2]:
Spherical = CS_spherical()
θ, ϕ, r = getcoords(Spherical) # Note the order of coordinates not `r, θ, ϕ` but `θ, ϕ, r` so that the frame `(𝐞ᶿ, 𝐞ᵠ, 𝐞ʳ)` coincides with the canonical frame for null angles
𝐞ᶿ, 𝐞ᵠ, 𝐞ʳ = unitvec(Spherical) # "\bfe<TAB>" to write bold `𝐞` and "\^\theta<TAB><TAB>" to write superscript `ᶿ`
𝐱 = getOM(Spherical) # Defines the current position vector in terms of spherical coordinates (ie `𝐱 = r 𝐞ʳ`)
𝐞₁, 𝐞₂, 𝐞₃ = unitvec(CS_cartesian()) ;

In [3]:
𝟏, 𝟙, 𝕀, 𝕁, 𝕂 = init_isotropic() # Defines usual isotropic Tensors
k, μ = symbols("k μ", positive = true)
λ = k -2μ/3 ;

## General resolution

### Hydrostatic loading

\begin{equation}
\mathbf{u}\underset{||\mathbf{x}||\to\infty}{\sim}\mathbf{E}\cdot\mathbf{x} \textrm{ with } \mathbf{E}=\frac{1}{3}E_v\mathbf{1}
\end{equation}

The displacement field is naturally searched in a form satisfying the isotropy of the loading ie
\begin{equation}
\mathbf{u}=u_r(r)\,\mathbf{e}_r
\end{equation}


In [4]:
u = SymFunction("u", real = true)
𝐮ˢᵖʰ = u(r) * 𝐞ʳ  # Note that the vector is in bold font ("\bfu<TAB>") and the component in normal font
𝛆ˢᵖʰ = tenssimp(SYMGRAD(𝐮ˢᵖʰ, Spherical)) # Strain tensor ("\bfepsilon<TAB>")

3×3 TensND.TensndRotated{2, 3, Sym, SymmetricTensor{2, 3, Sym, 6}}:
 u(r)/r       0                    0
      0  u(r)/r                    0
      0       0  Derivative(u(r), r)

In [5]:
𝛔ˢᵖʰ = tenssimp(λ * tr(𝛆ˢᵖʰ) * 𝟏 + 2μ * 𝛆ˢᵖʰ) # Stress tensor ("\bfsigma<TAB>")
𝐓ˢᵖʰ = tenssimp(𝛔ˢᵖʰ ⋅ 𝐞ʳ) ;

In [6]:
div𝛔ˢᵖʰ = DIV(𝛔ˢᵖʰ, Spherical) ;

In [7]:
eqˢᵖʰ = factor(simplify(div𝛔ˢᵖʰ ⋅ 𝐞ʳ))

            ⎛     2                              ⎞
            ⎜ 2  d              d                ⎟
(3⋅k + 4⋅μ)⋅⎜r ⋅───(u(r)) + 2⋅r⋅──(u(r)) - 2⋅u(r)⎟
            ⎜     2             dr               ⎟
            ⎝   dr                               ⎠
──────────────────────────────────────────────────
                          2                       
                       3⋅r                        

In [8]:
solˢᵖʰ = dsolve(eqˢᵖʰ, u(r)) ;

In [9]:
ûˢᵖʰ = solˢᵖʰ.rhs() ; display(ûˢᵖʰ)
T̂ˢᵖʰ = factor(simplify(subs(𝐓ˢᵖʰ ⋅ 𝐞ʳ, u(r) => ûˢᵖʰ))) ; display(T̂ˢᵖʰ)

C₁       
── + C₂⋅r
 2       
r        

                  3
-4⋅C₁⋅μ + 3⋅C₂⋅k⋅r 
───────────────────
          3        
         r         

### Deviatoric loading

\begin{equation}
\mathbf{u}\underset{||\mathbf{x}||\to\infty}{\sim}\mathbf{E}\cdot\mathbf{x} \textrm{ with } \mathbf{E}=E\,(\mathbf{e}_1\otimes\mathbf{e}_1+\mathbf{e}_2\,\otimes\mathbf{e}_2-2\mathbf{e}_3\,\otimes\mathbf{e}_3)=\mathbb{1}-3\mathbf{e}_3\otimes\mathbf{e}_3
\end{equation}

Note that such a macroscopic strain tensor induces a symmetry of revolution of the fields, which means in particular that the displacement field is expect of the form

\begin{equation}
\mathbf{u}=u_\theta(\theta,r)\,\mathbf{e}_\theta+u_r(\theta,r)\,\mathbf{e}_r
\end{equation}

In [10]:
𝐄 = 𝟏 - 3𝐞₃⊗𝐞₃
# Remote trends in θ of the displacement
fᶿ = 𝐞ᶿ ⋅ 𝐄 ⋅ 𝐞ʳ
fʳ = 𝐞ʳ ⋅ 𝐄 ⋅ 𝐞ʳ ;


In [11]:
uᶿ = SymFunction("uᶿ", real = true)
uʳ = SymFunction("uʳ", real = true)
𝐮ᵈᵉᵛ = uᶿ(r)* fᶿ * 𝐞ᶿ + uʳ(r)* fʳ * 𝐞ʳ
𝛆ᵈᵉᵛ = tenssimp(SYMGRAD(𝐮ᵈᵉᵛ, Spherical)) ;

In [12]:
𝛔ᵈᵉᵛ = tenssimp(λ * tr(𝛆ᵈᵉᵛ) * 𝟏 + 2μ * 𝛆ᵈᵉᵛ)
𝐓ᵈᵉᵛ = tenssimp(𝛔ᵈᵉᵛ ⋅ 𝐞ʳ) ;

In [13]:
div𝛔ᵈᵉᵛ = DIV(𝛔ᵈᵉᵛ, Spherical) ;

In [14]:
eqᶿᵈᵉᵛ = factor(simplify(div𝛔ᵈᵉᵛ ⋅ 𝐞ᶿ / fᶿ)) ;

In [15]:
eqʳᵈᵉᵛ = factor(simplify(div𝛔ᵈᵉᵛ ⋅ 𝐞ʳ / fʳ)) ;

In [16]:
α, Λ = symbols("α Λ", real = true)
eqᵈᵉᵛ = simplify.(subs.([eqᶿᵈᵉᵛ,eqʳᵈᵉᵛ], uᶿ(r) => r^α, uʳ(r) => Λ*r^α))

2-element Vector{Sym}:
                                   (6*k*r^α*Λ*α + 12*k*r^α*Λ - 18*k*r^α + 2*r^α*Λ*α*μ + 16*r^α*Λ*μ + 3*r^α*α*μ*(α - 1) + 6*r^α*α*μ - 24*r^α*μ)/(3*r^2)
 (3*k*r^α*Λ*α*(α - 1) + 6*k*r^α*Λ*α - 6*k*r^α*Λ - 9*k*r^α*α + 9*k*r^α + 4*r^α*Λ*α*μ*(α - 1) + 8*r^α*Λ*α*μ - 26*r^α*Λ*μ - 3*r^α*α*μ + 21*r^α*μ)/(3*r^2)

In [17]:
αΛ = solve(eqᵈᵉᵛ, [α, Λ])

4-element Vector{Tuple{Sym, Sym}}:
 (-4, -3/2)
 (-2, 3*(k + μ)/(2*μ))
 (1, 1)
 (3, 3*(3*k - 2*μ)/(15*k + 11*μ))

In [18]:
ûᶿᵈᵉᵛ = sum([Sym("C$(i+2)") * r^αΛ[i][1] for i ∈ 1:length(αΛ)]) ; display(ûᶿᵈᵉᵛ)
ûʳᵈᵉᵛ = sum([Sym("C$(i+2)") * αΛ[i][2] * r^αΛ[i][1] for i ∈ 1:length(αΛ)]) ; display(ûʳᵈᵉᵛ)

C₃   C₄              3
── + ── + C₅⋅r + C₆⋅r 
 4    2               
r    r                

                                     3            
  3⋅C₃   3⋅C₄⋅(k + μ)          3⋅C₆⋅r ⋅(3⋅k - 2⋅μ)
- ──── + ──────────── + C₅⋅r + ───────────────────
     4         2                   15⋅k + 11⋅μ    
  2⋅r       2⋅r ⋅μ                                

In [19]:
T̂ᶿᵈᵉᵛ = factor(simplify(subs(𝐓ᵈᵉᵛ ⋅ 𝐞ᶿ / fᶿ, uᶿ(r) => ûᶿᵈᵉᵛ, uʳ(r) => ûʳᵈᵉᵛ))) ; display(T̂ᶿᵈᵉᵛ)
T̂ʳᵈᵉᵛ = factor(simplify(subs(𝐓ᵈᵉᵛ ⋅ 𝐞ʳ / fʳ, uᶿ(r) => ûᶿᵈᵉᵛ, uʳ(r) => ûʳᵈᵉᵛ))) ; display(T̂ʳᵈᵉᵛ)


                     2          2  2            2              5            5 
-120⋅C₃⋅k⋅μ - 88⋅C₃⋅μ  + 45⋅C₄⋅k ⋅r  + 33⋅C₄⋅k⋅r ⋅μ + 30⋅C₅⋅k⋅r ⋅μ + 22⋅C₅⋅r ⋅
──────────────────────────────────────────────────────────────────────────────
                                                5                             
                                               r ⋅(15⋅k + 11⋅μ)               

 2            7            7  2
μ  + 48⋅C₆⋅k⋅r ⋅μ + 10⋅C₆⋅r ⋅μ 
───────────────────────────────
                               
                               

 ⎛                      2           2  2             2            2  2        
-⎝-180⋅C₃⋅k⋅μ - 132⋅C₃⋅μ  + 135⋅C₄⋅k ⋅r  + 159⋅C₄⋅k⋅r ⋅μ + 44⋅C₄⋅r ⋅μ  - 30⋅C₅
──────────────────────────────────────────────────────────────────────────────
                                                         5                    
                                                        r ⋅(15⋅k + 11⋅μ)      

    5            5  2           7           7  2⎞ 
⋅k⋅r ⋅μ - 22⋅C₅⋅r ⋅μ  + 9⋅C₆⋅k⋅r ⋅μ - 6⋅C₆⋅r ⋅μ ⎠ 
──────────────────────────────────────────────────
                                                  
                                                  